In [1]:
import cv2
import numpy as np
import pandas as pd
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from collections import Counter

from keras.callbacks import Callback
from keras.backend import clear_session
from keras.models import Model, load_model
from keras.layers import Dense, Input, Flatten
from keras.applications import ResNet50, MobileNet, Xception, DenseNet121

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
root_path = 'MyDrive/Recipes5k/annotations'

In [8]:
cd MyDrive/Recipes5k

[Errno 2] No such file or directory: '/MyDrive/Recipes5k'
/content


In [ ]:
## Downloading file from Google Drive

download = drive.CreateFile({'id': '1Kuz9LVt9nxFghTwDeo9csu0lnNdIbmu8'})
download.GetContentFile('data.tar.gz')

In [ ]:
!tar -xzvf data.tar.gz

tar (child): data.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
cd ./data/keras

/content/data/keras


In [ ]:
def load_data(df):
    
    trainX, testX, valX = [], [], []
    trainY, testY, valY = [], [], []
    
    for i in range(len(df)):
        
        item = df.loc[i][0]
        current_label = np.array((df.loc[i])[1:])
        
        path = os.path.join('images', item)
        list_of_imgs = [os.path.join(path, file) for file in os.listdir(path)]
        train_set = list_of_imgs[:30]
        val_set = list_of_imgs[30:40]
        test_set = list_of_imgs[40:]
        
        for file in train_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            trainX.append(img)
            trainY.append(current_label)
        
        for file in val_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            valX.append(img)
            valY.append(current_label)
        
        for file in test_set:
            img = cv2.resize(cv2.cvtColor(cv2.imread(file, 1), cv2.COLOR_BGR2RGB), (224, 224))
            testX.append(img)
            testY.append(current_label)
            
    return (np.array(trainX), np.array(trainY), np.array(testX), 
            np.array(testY), np.array(valX), np.array(valY))

In [ ]:
df = pd.read_csv('clean_anno_reduced.csv')
trainX, trainY, testX, testY, valX, valY = load_data(df)

In [ ]:
## Normalization

trainX = trainX.astype(np.float32)
testX = testX.astype(np.float32)
valX = valX.astype(np.float32)

trainY = trainY.astype(np.float32)
testY = testY.astype(np.float32)
valY = valY.astype(np.float32)

MEAN = np.mean(trainX, axis = (0,1,2))
STD = np.std(trainX, axis = (0,1,2))

for i in range(3):
    trainX[:, :, :, i] = (trainX[:, :, :, i] - MEAN[i]) / STD[i]
    testX[:, :, :, i] = (testX[:, :, :, i] - MEAN[i]) / STD[i]
    valX[:, :, :, i] = (valX[:, :, :, i] - MEAN[i]) / STD[i]

In [ ]:
MEAN, STD

(array([51.072815, 51.072815, 51.072815], dtype=float32),
 array([108.75629,  92.98068,  85.61884], dtype=float32))

In [ ]:
clear_session()
img = Input(shape = (224, 224, 3))

## You can replace DenseNet121 with MobileNet, Xception or ResNet50
model = DenseNet121(include_top=False, 
                    weights='imagenet', 
                    input_tensor=img, 
                    input_shape=None, 
                    pooling='avg')
final_layer = model.layers[-1].output

dense_layer_1 = Dense(128, activation = 'relu')(final_layer)
output_layer = Dense(10, activation = 'sigmoid')(dense_layer_1)

model = Model(input = img, output = output_layer)

In [ ]:
## Custom callback to compute F1 Score and save the model
## with best validation F1 Score.

class ComputeF1(Callback):
    
    def __init__(self):
        self.best_f1 = -1
        
    def on_epoch_end(self, epoch, logs={}):
        val_pred = np.round(self.model.predict(self.validation_data[0]))
        val_f1 = f1_score(self.validation_data[1], val_pred, average='samples')
        print('Validation Average F1 Score: ', val_f1)
        
        if val_f1 > self.best_f1:
            print('Better F1 Score, Saving model...')
            self.model.save('model.h5')
            self.best_f1 = val_f1

In [ ]:
f1_score_callback = ComputeF1()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy' , metrics = ['accuracy'])
#model.summary()

In [ ]:
model.fit(trainX, trainY, batch_size = 32, epochs = 25, validation_data = (valX, valY), 
          callbacks = [f1_score_callback])

In [ ]:
## Loading best model based on validation score

from keras.models import load_model
model = load_model('model.h5')

In [ ]:
def F1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='samples')

In [ ]:
F1_score(testY, model.predict(testX).round())

0.7064709348159105